# Формирование риск-факторов для Контрагентов сегмента "Крупнейшие заемщики" (TOP, компании и группы)

<br>
  
# Содержание <a name=content></a>
  
<br>

- [<span style="color:red">**Версионность**</span>](#versions)
- [<span style="color:red">**Конфигурационные файлы**</span>](#config_files)
<br>
  
- [<span style="color:brown">**Импорт библиотек + Мастер-Шкала + Словарь**</span>](#initial_import)
- [Определение функций](#functions)

<br>

**ЧАСТЬ I   -- ЗАГРУЗКА ДАННЫХ**
___

  
- [**Загрузка данных**](#upld_data)
    - [Макроэкономические данные](#upld_macro)
    - [Определение целевой переменной](#define_target)
  
  
- [**EDA** (Exploratory Data Analysis)](#eda)
    - [Расчёт статистик (+гистограммы распределений)](#statistics_histogramms)
    
    
<br>

**ЧАСТЬ II  -- Features Engineering**
___
    
- [Риск-факторы Финансовые](#factors_qn)   
    - [**Scale**](#factors_qn_scale)
    - [**Leverage**](#factors_qn_leverage)
    - [**Liquidity**](#factors_qn_liquidity)
    - [**Profitability**](#factors_qn_profitability)

    
    
- [<span style="color:green">**DEVELOPMENT Sample**</span>](#dev_sample)  

<br>

**ЧАСТЬ III -- SFA**
___
    

- [SFA (Single Factor Analysis)](#SFA)

  
  
<br>

**ЧАСТЬ IV -- MFA**
___
    
  
- [MFA - финансовый модуль](#MFA_qn)

  
  
<br>

**ЧАСТЬ V  -- Калибровка**
___
    
  
- [Калибровка](#Calibration)

  
  
<br>

**ЧАСТЬ VI -- Валидация**
___
    
  
- [Валидация модели](#Validation)

<br>
  
  
# Версионность <a name=versions></a>
  
<br>

In [1]:
c_version_r = "050_1"
c_version_w = "050_2"

from IPython.core.display import display, HTML
display(HTML("<style>.container {width:85% !important; }</style>"))

In [2]:
%%javascript

Jupyter.keyboard_manager.disable()

<IPython.core.display.Javascript object>

<br>


# Конфигурационные файлы <a name=config_files></a>

[Содержание](#content)

<br>


# Импорт библиотек + Master Scale + Словарь <a name= initial_import ></a>

[Содержание](#content)

In [2]:
def RATINGS_func(x,CALIB):
    
    #CALIB = pd.read_csv("D:\\Shepitko-IS\\Desktop\\Projects\\a6_2_DenizBank\\Group_Government_Support\\a6_2_DENIZ1819_CALIB_v0_1.csv",delimiter=';',skipinitialspace=True)
    
    # masterscale
    MS_PD = [ 0.00020, 0.00037, 0.00051, 0.00070, 0.00097, 0.00133, 0.00184, 0.00253, 0.00348, 0.00480, 0.00660
             ,0.00910, 0.01253, 0.01725, 0.02375, 0.03271, 0.04505, 0.06204, 0.08543, 0.11765, 0.16203, 0.22313
             ,0.30728, 0.42316, 0.58275, 1 ]
    
    MS_LB = [ 0.00000, 0.00032, 0.00044, 0.00060, 0.00083, 0.00114, 0.00156, 0.00215, 0.00297, 0.00409, 0.00563
             ,0.00775, 0.01067, 0.01470, 0.02024, 0.02788, 0.03839, 0.05287, 0.07280, 0.10026, 0.13807, 0.19014
             ,0.26185, 0.36059, 0.49659, 1 ]
    
    MS_UB = [ 0.00032, 0.00044, 0.00060, 0.00083, 0.00114, 0.00156, 0.00215, 0.00297, 0.00409, 0.00563, 0.00775
             ,0.01067, 0.01470, 0.02024, 0.02788, 0.03839, 0.05287, 0.07280, 0.10026, 0.13807, 0.19014, 0.26185
             ,0.36059, 0.49659, 1, 1 ]
    
    
    
    CALIB['PD_opt'] = 1/(1+np.exp(-(x[0]*CALIB['BASE_score']+x[1])))
    
    for j in range(0,26):
        indx1 = CALIB['PD_opt'] >= float(MS_LB[j])
        indx2 = CALIB['PD_opt'] <  float(MS_UB[j])
        
        CALIB.loc[indx1 & indx2,'Rating'] = j+1
        CALIB.loc[indx1 & indx2,'PD_mapback'] = float(MS_PD[j])    
         
    return CALIB

<br>
  
  
# Определение функций <a name= functions></a>
  
  
[Содержание](#content)

<br>
  
  
# <span style="color:Indigo"> ЗАГРУЗКА ДАННЫХ </span>


<br>

# UPLOAD dev samples

In [45]:
portfolio = pd.read_csv("C:\\Users\\ivanmipt\\Desktop\\Socio\\d___Investments\\Data\\Portfolios\\DenizPortfolio.txt",encoding='windows-1251',delimiter="\t")
print(portfolio.dtypes)
print(portfolio.shape)

approval_date     object
customer_no        int64
appr_VINTAGE      object
default            int64
weight           float64
BASE_score       float64
dtype: object
(11786, 6)


In [46]:
from datetime import datetime

portfolio['VINTAGE'] = portfolio['appr_VINTAGE'].map(lambda x: datetime.strptime(x, '%d%b%Y'))
portfolio = portfolio[['default','weight','BASE_score','VINTAGE']]

In [47]:
x = [-0.0526 ,-5.1109  ]
portfolio_u = RATINGS_func(x,portfolio)

## define loiville's empirical velocities

**There are next types of factors:**

USD liquid positive affect <br/>
USD liquid negative affect <br/>
USD non-liquid positive affect <br/>
USD non-liquid negative affect <br/>
<br/>
<br/>
RUR liquid positive affect <br/>
RUR liquid negative affect <br/>
RUR non-liquid positive affect <br/>
RUR non-liquid negative affect <br/>

In [54]:
for i in range(0,len(dates)-1):
    
    dat_t = dates[i]
    dat_t_1 = dates[i+1]
    
    tmp_t = pd.DataFrame(sample_u[sample_u['VINTAGE']==dat_t])
    tmp_t_1 = pd.DataFrame(sample_u[sample_u['VINTAGE']==dat_t_1])
    tmp = pd.DataFrame(data = { 'VINTAGE' : [dat_t] })
    
    for j in range(0,22):
        
        if j==0 :
            tmp['u_Rating_0'] = 0.0
        else:
            nam2 = 'distr_Rating_' + str(j)
            nam2_ = 'distr_Rating_' + str(j-1)
            
            nam3 = 'u_Rating_' + str(j)
            nam3_ = 'u_Rating_' + str(j-1)
            
            #print('a------------1')
            #print(float(tmp[nam3_]))
            #print(float(tmp_t[nam2_]))
            #print(float(tmp_t[nam2_]))
            
            #print(float(tmp_t_1[nam2_]))
            #print(float(tmp_t[nam2]))
            
            if float(tmp_t[nam2]) > 0:
                tmp[nam3] = round(((  float(tmp[nam3_])*float(tmp_t[nam2_])
                         -(float(tmp_t[nam2_])-float(tmp_t_1[nam2_])))/float(tmp_t[nam2])),4)
            else:
                tmp[nam3] = 0.0
                
    if i==0:
        fin = tmp
    else:
        fin = fin.append([tmp])
    

# Kinetic approach

### the general louisville's equation:

![title](Pictures/d___001.png)

### the expression that binds velocity and empirical distribution function:

![title](Pictures/d___002.png)

In [163]:
#USD
'''
usd_liq_pos_P = pd.DataFrame(X_PASSIVE[X_PASSIVE['NUM_SC'].isin(l_usd_liq_pos_P)].groupby('DT')['IV'].sum().reset_index())
usd_liq_pos_A = pd.DataFrame(X_ACTIVE[X_ACTIVE['NUM_SC'].isin(l_usd_liq_pos_A)].groupby('DT')['IV'].sum().reset_index())

usd_liq_pos = usd_liq_pos_P.merge(usd_liq_pos_A, how='inner', on='DT')
usd_liq_pos['var1'] = usd_liq_pos['IV_x']/usd_liq_pos['IV_y']

usd_liq_neg_P = pd.DataFrame(X_PASSIVE[X_PASSIVE['NUM_SC'].isin(l_usd_liq_neg_P)].groupby('DT')['IV'].sum().reset_index())
usd_liq_neg_A = pd.DataFrame(X_ACTIVE[X_ACTIVE['NUM_SC'].isin(l_usd_liq_neg_A)].groupby('DT')['IV'].sum().reset_index())

usd_liq_neg = usd_liq_neg_P.merge(usd_liq_neg_A, how='inner', on='DT')
usd_liq_neg['var2'] = usd_liq_neg['IV_x']/usd_liq_neg['IV_y']
'''


usd_nliq_pos_P = pd.DataFrame(X_PASSIVE[X_PASSIVE['NUM_SC'].isin(l_usd_nliq_pos_P)].groupby('DT')['IV'].sum().reset_index())
usd_nliq_pos_A = pd.DataFrame(X_ACTIVE[X_ACTIVE['NUM_SC'].isin(l_usd_nliq_pos_A)].groupby('DT')['IV'].sum().reset_index())

usd_nliq_pos = usd_nliq_pos_P.merge(usd_nliq_pos_A, how='inner', on='DT')
usd_nliq_pos['var3'] = usd_nliq_pos['IV_x']/usd_nliq_pos['IV_y']

usd_nliq_neg_P = pd.DataFrame(X_PASSIVE[X_PASSIVE['NUM_SC'].isin(l_usd_nliq_neg_P)].groupby('DT')['IV'].sum().reset_index())
usd_nliq_neg_A = pd.DataFrame(X_ACTIVE[X_ACTIVE['NUM_SC'].isin(l_usd_nliq_neg_A)].groupby('DT')['IV'].sum().reset_index())

usd_nliq_neg = usd_nliq_neg_P.merge(usd_nliq_neg_A, how='inner', on='DT')
usd_nliq_neg['var4'] = -usd_nliq_neg['IV_x']/usd_nliq_neg['IV_y']


In [164]:
#RUR

'''


rur_liq_pos_P = pd.DataFrame(X_PASSIVE[X_PASSIVE['NUM_SC'].isin(l_rur_liq_pos_P)].groupby('DT')['IV'].sum().reset_index())
rur_liq_pos_A = pd.DataFrame(X_ACTIVE[X_ACTIVE['NUM_SC'].isin(l_rur_liq_pos_A)].groupby('DT')['IV'].sum().reset_index())

rur_liq_pos = rur_liq_pos_P.merge(rur_liq_pos_A, how='inner', on='DT')
rur_liq_pos['var5'] = rur_liq_pos['IV_x']/rur_liq_pos['IV_y']

rur_liq_neg_P = pd.DataFrame(X_PASSIVE[X_PASSIVE['NUM_SC'].isin(l_rur_liq_neg_P)].groupby('DT')['IV'].sum().reset_index())
rur_liq_neg_A = pd.DataFrame(X_ACTIVE[X_ACTIVE['NUM_SC'].isin(l_rur_liq_neg_A)].groupby('DT')['IV'].sum().reset_index())

rur_liq_neg = rur_liq_neg_P.merge(rur_liq_neg_A, how='inner', on='DT')
rur_liq_neg['var6'] = rur_liq_neg['IV_x']/rur_liq_neg['IV_y']

'''


rur_nliq_pos_P = pd.DataFrame(X_PASSIVE[X_PASSIVE['NUM_SC'].isin(l_rur_nliq_pos_P)].groupby('DT')['IR'].sum().reset_index())
rur_nliq_pos_A = pd.DataFrame(X_ACTIVE[X_ACTIVE['NUM_SC'].isin(l_rur_nliq_pos_A)].groupby('DT')['IR'].sum().reset_index())

rur_nliq_pos = rur_nliq_pos_P.merge(rur_nliq_pos_A, how='inner', on='DT')
rur_nliq_pos['var7'] = rur_nliq_pos['IR_x']/rur_nliq_pos['IR_y']

rur_nliq_neg_P = pd.DataFrame(X_PASSIVE[X_PASSIVE['NUM_SC'].isin(l_rur_nliq_neg_P)].groupby('DT')['IR'].sum().reset_index())
rur_nliq_neg_A = pd.DataFrame(X_ACTIVE[X_ACTIVE['NUM_SC'].isin(l_rur_nliq_neg_A)].groupby('DT')['IR'].sum().reset_index())

rur_nliq_neg = rur_nliq_neg_P.merge(rur_nliq_neg_A, how='inner', on='DT')
rur_nliq_neg['var8'] = -rur_nliq_neg['IR_x']/rur_nliq_neg['IR_y']


In [165]:
#drop
usd_nliq_pos.drop(['IV_x','IV_y'],axis=1,inplace=True)
usd_nliq_neg.drop(['IV_x','IV_y'],axis=1,inplace=True)

rur_nliq_pos.drop(['IR_x','IR_y'],axis=1,inplace=True)
rur_nliq_neg.drop(['IR_x','IR_y'],axis=1,inplace=True)

### training dataset

In [166]:
from functools import reduce

parts = [usd_nliq_pos, usd_nliq_neg, rur_nliq_pos, rur_nliq_neg]
final = reduce(lambda left,right: pd.merge(left,right,on='DT'), parts)

In [167]:
target = target_init[['net capital flow','DT']]
target.columns = ['target','DT']

In [168]:
TRAIN_TEST = pd.merge(final,target,how='inner',on='DT')

for i in range(1,len(TRAIN_TEST.columns)-1):
    nam = TRAIN_TEST.columns[i]
    t1 = TRAIN_TEST[nam]
    t2 = TRAIN_TEST['target']
    c = np.corrcoef(t1, t2)
    
    print('\n','for ',nam,' : ',round(c[0][1],4),'\n')
    


 for  var3  :  0.6314 


 for  var4  :  0.5595 


 for  var7  :  0.6217 


 for  var8  :  0.5214 



## correlation analysis

# Fitting the Model

In [169]:
reg = LinearRegression()

TRAIN = TRAIN_TEST[['var3','var4','var7','var8']] #.values.reshape(-1,1)
y = TRAIN_TEST['target']

reg.fit(TRAIN,y)

pred = reg.predict(TRAIN)
pred_  = pd.DataFrame(pred)

In [170]:
c = reg.coef_

for j in range(0,len(c)):
    print('koef ',j+1,' : ',round(c[j],2),'\n')

cc = np.corrcoef(pred,y)[0,1]
    
sd = Somers_D(y,pred)

print("cc: ",cc,"  somD: ",sd)

koef  1  :  0.06 

koef  2  :  -0.34 

koef  3  :  4.37 

koef  4  :  134.3 

cc:  0.7454423243890042   somD:  0.5616326530612245


In [66]:
pred_vs_fact = pd.DataFrame([pred,y]).transpose()
pred_vs_fact

,0,1
0,16.947476,19.0
1,13.334611,14.3
2,11.667038,14.1
3,8.913794,25.0
4,8.522760,9.2
5,2.440124,-12.7
6,11.883256,6.4
7,12.447030,1.2
8,13.961172,28.5
9,15.438153,13.3


In [ ]:
for_h.plot(x='Rating',y='distr',kind='bar',color='b')
for_h.PD.plot(secondary_y = True, linestyle='-',marker='o',color='r')

plt.savefig('D:\\Shepitko-IS\\Desktop\\Projects\\b5___Sub_Souvereigns\\Data\\b5___final_distribution.png')



In [ ]:
model_res.groupby('Rating')['PD_obs'].count()

In [ ]:
y = TRAIN_fin['PD']
X = TRAIN_fin.drop('PD',axis=1)

In [ ]:
logistic_regression_model = LogisticRegression(penalty='l1',C=0.75,solver='saga',multi_class='multinomial')

log_reg_model = logistic_regression_model.fit(X,y);
print('Коэффициенты наклона бета =',log_reg_model.coef_[0,:])

print('----------------------------------')
Norm_vesa_log_reg_reg = abs(log_reg_model.coef_)/nm.repmat(sum(abs(log_reg_model.coef_[0,:])),1,np.size(log_reg_model.coef_))
print("Нормированные веса для модели логистической регрессии с регуляризацией =",Norm_vesa_log_reg_reg[0,:])

print('----------------------------------')
print('Коэффициент альфа =',log_reg_model.intercept_[0])